<a href="https://colab.research.google.com/github/C4viar/PPD_MSLD_2019_2020_LL-AL-MT/blob/master/Draft_code/projet_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contexte


Des modèles de langage du français ont tout récemment été publiés (CamemBERT, FlauBERT C’est positif car même des outils avancés comme Spacy sont notoirement encore peu adaptés à traiter certaines langues dont le français. Il reste à évaluer ce que ces modèles peuvent vraiment apporter notamment ne ce qui concerne la reconnaissances d’entités nommées.


# Travail a réaliser

Réaliser des expériences de reconnaissance d’entités nommées sur le jeu de donneés Europeana Newspapers French en utilisant tous les modèles disponibles pour cela dans la bibliothèque Simple Transformers. Les adresses à consulter sont :
* https://github.com/ThilinaRajapakse/simpletransformers
* http://api.bnf.fr/europeana-newspapers-french-ner-ground-truth

# Rendu

Un code clair et commenté dans un notebook.
Un rapport décrivant les méthodes/métriques d’évaluation (basées sur seqeval dont une description détaillée sera donnée) et présentant/comparant/discutant les résultats obtenus.


# Seqeval 

https://pypi.org/project/seqeval/

"seqeval is a Python framework for sequence labeling evaluation. seqeval can evaluate the performance of chunking tasks such as named-entity recognition, part-of-speech tagging, semantic role labeling and so on."

# Installation simple transformers

In [0]:
! pip install simpletransformers

# NER 
https://simpletransformers.ai/docs/ner-specifics/

"The goal of Named Entity Recognition is to locate and classify named entities in a sequence. The named entities are pre-defined categories chosen according to the use case such as names of people, organizations, places, codes, time notations, monetary values, etc. Essentially, NER aims to assign a class to each token (usually a single word) in a sequence. Because of this, NER is also referred to as token classification."

**Fonctionnement avec Simple Transformer**

* Initialize a NERModel
* Train the model with train_model()
* Evaluate the model with eval_model()
* Make predictions on (unlabelled) data with predict()

**Modèles disponibles à fournir à model_type**

* BERT :	*bert*
* CamemBERT :	*camembert*
* RoBERTa :	*roberta*
* DistilBERT :	*distilbert*
* ELECTRA	: *electra*
* XLM-RoBERTa	: *xlmroberta*

In [3]:
from simpletransformers.classification import NERModel

# to custom labels
custom_labels = ["O", "B-SPELL", "I-SPELL", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-PLACE", "I-PLACE"]

model = NERModel(
    model_type="bert", model_name="bert-cased-base", labels=custom_labels
)

ImportError: ignored